# Build a Multi-Layer Neural Network


## Weights Initialization
---
Firstly, weights need to be initialized for different layers. Note that in general, the input is not considered as a layer, but output is.

For `lth` layer, 
- weight $W^{[l]}$ has shape $(n^{[l]}, n^{[l-1]})$
- bias $b^{[l]}$ has shape $(n^{[l]}, 1)$

where $n^{[0]}$ equals the number input feature.

In [1]:
import numpy as np

In [2]:
def weights_init(layers_dim):
    params = {}
    
    n = len(layers_dim)
    for i in range(1, n):
        params['W' + str(i)] = np.random.randn(layers_dim[i], layers_dim[i-1])*0.01
        params['b' + str(i)] = np.zeros((layers_dim[i], 1))
    return params

In [3]:
p = weights_init([2, 5, 1])
p

{'W1': array([[ 0.00578096, -0.0033889 ],
        [-0.00802873, -0.00708873],
        [-0.01977247,  0.00021493],
        [ 0.00947858,  0.00554905],
        [-0.01050366,  0.0114642 ]]),
 'b1': array([[0.],
        [0.],
        [0.],
        [0.],
        [0.]]),
 'W2': array([[ 0.00446584,  0.0011277 ,  0.00521519, -0.00092573,  0.00270877]]),
 'b2': array([[0.]])}

# Forward
---
## Equations of Multi-layer
---
$$ Z^{[l]} = W^{[l]}A^{[l-1]} + b^{[l]} $$

$$ A^{[l]} = g^{[l]}(Z^{[l]}) $$

Where $l$ is the `lth` layer.

In [4]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))


def relu(x):
    return np.maximum(x, 0)

In [5]:
x = np.array([-1.2, -2.0, 1.3])

sx = sigmoid(x)
rx = relu(x)

print(sx, rx)

[0.23147522 0.11920292 0.78583498] [0.  0.  1.3]


In [6]:
def forward(X, params):
    # intermediate layer use relu as activation
    # last layer use sigmoid
    n_layers = int(len(params)/2)
    A = X
    cache = {}
    for i in range(1, n_layers):
        W, b = params['W'+str(i)], params['b'+str(i)]
        Z = np.dot(W, A) + b
        A = relu(Z)
        cache['Z'+str(i)] = Z
        cache['A'+str(i)] = A
    
    # last layer
    W, b = params['W'+str(i+1)], params['b'+str(i+1)]
    Z = np.dot(W, A) + b
    A = sigmoid(Z)
    cache['Z'+str(i+1)] = Z
    cache['A'+str(i+1)] = A
    
    return cache, A

In [7]:
X = np.array([1., 1.]).reshape(2, 1)
cache, A = forward(X, p)

In [8]:
cache

{'Z1': array([[ 0.00239207],
        [-0.01511746],
        [-0.01955754],
        [ 0.01502763],
        [ 0.00096054]]),
 'A1': array([[0.00239207],
        [0.        ],
        [0.        ],
        [0.01502763],
        [0.00096054]]),
 'Z2': array([[-6.27109322e-07]]),
 'A2': array([[0.49999984]])}

In [9]:
A

array([[0.49999984]])

# Cost Function
---
Still we consider this a binary classification, the cost of a batch would be:

$$-\frac{1}{m} \sum\limits_{i = 1}^{m} (y^{(i)}\log\left(a^{[L] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right)) $$

Where $a$ is the predicted value, and $y$ is the actual one.

In [10]:
def compute_cost(A, Y):
    """
    For binary classification, both A and Y would have shape (1, m), where m is the batch size
    """
    assert A.shape == Y.shape
    m = A.shape[1]
    s = np.dot(Y, np.log(A.T)) + np.dot(1-Y, np.log((1 - A).T))
    loss = -s/m
    return np.squeeze(loss)

In [11]:
A = np.array([[0.9, 0.3]])
Y = np.array([[1, 0]])

loss = compute_cost(A, Y)
print(loss)

0.23101772979827936


# Backward Propagation
---
<img src='images/backprop_kiank.png' style="width:800px;height:250px;">
<caption><center> **[source]**: https://github.com/enggen/Deep-Learning-Coursera </center></caption>

The backward gradient can be calculated in recurrent fashion:

$$ dZ^{[l]} = dA^{[l]} * g^{[l]'}(Z^{[l]}) $$

$$ dW^{[l]} = \frac{\partial \mathcal{L} }{\partial W^{[l]}} = \frac{1}{m} dZ^{[l]} A^{[l-1] T} $$
$$ db^{[l]} = \frac{\partial \mathcal{L} }{\partial b^{[l]}} = \frac{1}{m} \sum_{i = 1}^{m} dZ^{[l](i)}$$
$$ dA^{[l-1]} = \frac{\partial \mathcal{L} }{\partial A^{[l-1]}} = W^{[l] T} dZ^{[l]} $$


First, implementation of derivative of `sigmoid` and `relu` is required.

In [12]:
def sigmoid_grad(A, Z):
    grad = np.multiply(A, 1-A)
    return grad


def relu_grad(A, Z):
    grad = np.zeros(Z.shape)
    grad[Z>0] = 1
    return grad

In [13]:
z = np.random.randn(3, 2)
a = sigmoid(z)
g = sigmoid_grad(a, z)

print(z, '\n')
print(a, '\n')
print(g)

[[-0.90176825 -0.719363  ]
 [-2.60503925  1.01777905]
 [ 0.62077097  0.10424591]] 

[[0.28868726 0.32753327]
 [0.06881481 0.73453976]
 [0.65039387 0.5260379 ]] 

[[0.20534692 0.22025523]
 [0.06407933 0.1949911 ]
 [0.22738168 0.24932203]]


In [14]:
z = np.random.randn(3, 2)
a = relu(z)
g = relu_grad(a, z)

print(z, '\n')
print(a, '\n')
print(g)

[[ 0.08444414 -1.35978537]
 [ 0.58341162 -0.52831323]
 [ 1.48930994  1.84182838]] 

[[0.08444414 0.        ]
 [0.58341162 0.        ]
 [1.48930994 1.84182838]] 

[[1. 0.]
 [1. 0.]
 [1. 1.]]


Following the equations above, we have our implementation of backward propagation. Note that except the last layer where `sigmoid` function is used, the rest we all apply `relu` derivative to get the gradients.

In [15]:
def backward(params, cache, X, Y):
    """
    params: weight [W, b]
    cache: result [A, Z]
    Y: shape (1, m)
    """
    grad = {}
    n_layers = int(len(params)/2)
    m = Y.shape[1]
    cache['A0'] = X
    
    for l in range(n_layers, 0, -1):
        A, A_prev, Z = cache['A' + str(l)], cache['A' + str(l-1)], cache['Z' + str(l)]
        W = params['W'+str(l)]
        if l == n_layers:
            dA = -np.divide(Y, A) + np.divide(1 - Y, 1 - A)
        
        if l == n_layers:
            dZ = np.multiply(dA, sigmoid_grad(A, Z))
        else:
            dZ = np.multiply(dA, relu_grad(A, Z))
        dW = np.dot(dZ, A_prev.T)/m
        db = np.sum(dZ, axis=1, keepdims=True)/m
        dA = np.dot(W.T, dZ)

        grad['dW'+str(l)] = dW
        grad['db'+str(l)] = db
    
    return grad

In [16]:
g = backward(p, cache, np.array([[1], [1]]), np.array([[1]]))
g

{'dW2': array([[-0.00119603,  0.        ,  0.        , -0.00751382, -0.00048027]]),
 'db2': array([[-0.50000016]]),
 'dW1': array([[-0.00223292, -0.00223292],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.00046287,  0.00046287],
        [-0.00135439, -0.00135439]]),
 'db1': array([[-0.00223292],
        [ 0.        ],
        [ 0.        ],
        [ 0.00046287],
        [-0.00135439]])}

Now given the gradients, we have our weights updated as following:
    
$$ W^{[l]} -= dW^{[l]} $$
$$ b^{[l]} -= db^{[l]} $$

In [17]:
def optimize(params, grads, lr):
    n_layers = int(len(params)/2)
    for i in range(1, n_layers+1):
        dW, db = grads['dW'+str(i)], grads['db'+str(i)]
        params['W'+str(i)] -= lr*dW
        params['b'+str(i)] -= lr*db
    return params

In [18]:
s = optimize(p, g, 1)
s

{'W1': array([[ 0.00801388, -0.00115598],
        [-0.00802873, -0.00708873],
        [-0.01977247,  0.00021493],
        [ 0.00901571,  0.00508619],
        [-0.00914928,  0.01281859]]),
 'b1': array([[ 0.00223292],
        [ 0.        ],
        [ 0.        ],
        [-0.00046287],
        [ 0.00135439]]),
 'W2': array([[0.00566187, 0.0011277 , 0.00521519, 0.00658809, 0.00318904]]),
 'b2': array([[0.50000016]])}

# Apply on Dataset
---
Let's have our model apply on created dataset with 200 features.

In [19]:
from sklearn import datasets


X, y = datasets.make_classification(n_samples=10000, n_features=200, random_state=123)

X_train, X_test = X[:8000], X[8000:]
y_train, y_test = y[:8000], y[8000:]

print('train shape', X_train.shape)
print('test shape', X_test.shape)

train shape (8000, 200)
test shape (2000, 200)


In [20]:
def generate_batch(X, batch_size):
    n = X.shape[0]
    batches = [range(i, i+batch_size) for i in range(0, n, batch_size)]
    return batches


def accuracy(Y, Y_pred):
    """
    Y: vector of true value
    Y_pred: vector of predicted value
    """
    
    assert Y.shape[0] == 1
    assert Y.shape == Y_pred.shape
    Y_pred = np.round(Y_pred)
    acc = float(np.dot(Y, Y_pred.T) + np.dot(1 - Y, 1 - Y_pred.T))/Y.size
    return acc

In [21]:
def train(X_train, y_train, layers: list, batch_size=200, n_iter=100, lr=0.1):
    # prepare batch training
    batches = generate_batch(X_train, batch_size)
    # init weights
    parameters = weights_init(layers)
    for i in range(n_iter):
        for batch in batches:
            X = X_train[batch, :].T
            Y = y_train[batch].reshape(1, -1)
            cache, A = forward(X, parameters)
            grads = backward(parameters, cache, X, Y)
            parameters = optimize(parameters, grads, lr)
            
        if i%10 == 0:
            loss = compute_cost(A, Y)
            print(f'iteration {i}: loss {loss}')
    return parameters

Let's build a 3-layer neural network, with input of 200 features.

In [22]:
trained_param = train(X_train, y_train, layers=[200, 20, 10, 1], batch_size=200, n_iter=60, lr=0.1)

iteration 0: loss 0.6930634457206598
iteration 10: loss 0.693022813093077
iteration 20: loss 0.6930210137413801
iteration 30: loss 0.692971329136883
iteration 40: loss 0.6804278680112462
iteration 50: loss 0.17899414276926578


In [23]:
cache, pred = forward(X_test.T, trained_param)

In [24]:
acc = accuracy(y_test.reshape(1, -1), pred)

print(f'accuracy: {acc*100}%')

accuracy: 94.05%
